# Children in New York Lead levels in their blood 
In this notebook I first read in the data, dropped columns that did not have testable data in them, then split the data into Training and testing sets to check the accuracy of the following question:
    Does the Year of Birth affect whether or not a child would have levels of lead in their system? 

In [72]:
# Dependencies
import numpy as np
import pandas as pd
import datetime as dt
import requests
import json
from pprint import pprint
from matplotlib import style
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import RandomOverSampler

In [33]:
#  Import and read the combined csv
bt_df=pd.read_csv(Path('../brittany_analysis/data/cleaned_blood_data.csv'))
bt_df.tail()

,County,County Code,Zip,Year,Year of Birth,Tests,Less than 5 mcg/dL,5-10 mcg/dL,10-15 mcg/dL,15+ mcg/dL,Total Elevated Blood Levels,"Rate per 1,000",Percent,Zip Code Location,County Location
1812,Wyoming,121,14591,2020,2019,7,7.0,0.0,0.0,0.0,0.0,0.0,0.0,"(42.827666, -78.08667)","(42.701363, -78.228567)"
1813,Yates,123,14527,2020,2017,13,13.0,0.0,0.0,0.0,0.0,0.0,0.0,"(42.658509, -77.061185)","(42.6431, -77.1485)"
1814,Yates,123,14527,2020,2020,7,7.0,0.0,0.0,0.0,0.0,0.0,0.0,"(42.658509, -77.061185)","(42.6431, -77.1485)"
1815,Yates,123,14544,2020,2018,8,8.0,0.0,0.0,0.0,0.0,0.0,0.0,"(42.742889, -77.245455)","(42.6431, -77.1485)"
1816,Yates,123,14544,2020,2019,7,7.0,0.0,0.0,0.0,0.0,0.0,0.0,"(42.742889, -77.245455)","(42.6431, -77.1485)"


In [34]:
#Drop specific columns that are not needed 
bt_df=bt_df.drop(columns=['County','Zip Code Location','County Location'])


In [35]:
#Select your features (columns)
bt_df.columns

Index(['County Code', 'Zip', 'Year', 'Year of Birth', 'Tests',
       'Less than 5 mcg/dL', '5-10 mcg/dL', '10-15 mcg/dL', '15+ mcg/dL',
       'Total Elevated Blood Levels', 'Rate per 1,000', 'Percent'],
      dtype='object')

In [36]:
bt_df

,County Code,Zip,Year,Year of Birth,Tests,Less than 5 mcg/dL,5-10 mcg/dL,10-15 mcg/dL,15+ mcg/dL,Total Elevated Blood Levels,"Rate per 1,000",Percent
0,1,12009,2020,2019,30,30.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,12023,2020,2019,7,7.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,12054,2020,2017,6,6.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,12083,2020,2018,6,6.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,12083,2020,2019,7,7.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1812,121,14591,2020,2019,7,7.0,0.0,0.0,0.0,0.0,0.0,0.0
1813,123,14527,2020,2017,13,13.0,0.0,0.0,0.0,0.0,0.0,0.0
1814,123,14527,2020,2020,7,7.0,0.0,0.0,0.0,0.0,0.0,0.0
1815,123,14544,2020,2018,8,8.0,0.0,0.0,0.0,0.0,0.0,0.0


# Split the Data into Training and Testing Sets
Use 'Year of Birth' for the y value to determine if age is a factor or not

In [56]:
#Set features that will be used as our y and x values
y=bt_df['Year of Birth']
X=bt_df.drop(columns='Year of Birth')

In [57]:
#Review the y variable 
y[:5]

0    2019
1    2019
2    2017
3    2018
4    2019
Name: Year of Birth, dtype: int64

In [58]:
#Review the X variable 
X[:5]

,County Code,Zip,Year,Tests,Less than 5 mcg/dL,5-10 mcg/dL,10-15 mcg/dL,15+ mcg/dL,Total Elevated Blood Levels,"Rate per 1,000",Percent
0,1,12009,2020,30,30.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,12023,2020,7,7.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,12054,2020,6,6.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,12083,2020,6,6.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,12083,2020,7,7.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
#Check the balance of the targeted value 
y.value_counts()

2018    612
2019    425
2017    306
2016    183
2015    143
2020     98
2014     50
Name: Year of Birth, dtype: int64

In [60]:
#create a Train Test Split 
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

## Logistical Regression Model

In [61]:
#Create a Logistic Regression Model by using the training data 
lr_model=LogisticRegression(random_state=42)
#Fit the model using the training Data 
training_model=lr_model.fit(X_train,y_train)

c:\Users\britt\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [62]:
#make a prediction using testing data 
testing_predictions=training_model.predict(X_test)

In [63]:
#Show the balanced accuracy score of the model
balanced_model=balanced_accuracy_score(y_test, testing_predictions) 
print(balanced_model)

0.15967719741304648


In [64]:
# Generate a confusion matrix for the model
matrix_model=confusion_matrix(y_test,testing_predictions)
print(matrix_model)

[[  0   0   0   1   9   1   0]
 [  0   0   0   5  22   5   0]
 [  0   0   0  10  34   3   0]
 [  0   0   0   9  67   5   0]
 [  0   0   0   4 142   2   0]
 [  0   0   0   8  93   5   0]
 [  0   0   0   5  22   3   0]]


In [65]:
# Print the classification report for the model
classification_model=classification_report(y_test,testing_predictions)
print(classification_model)

              precision    recall  f1-score   support

        2014       0.00      0.00      0.00        11
        2015       0.00      0.00      0.00        32
        2016       0.00      0.00      0.00        47
        2017       0.21      0.11      0.15        81
        2018       0.37      0.96      0.53       148
        2019       0.21      0.05      0.08       106
        2020       0.00      0.00      0.00        30

    accuracy                           0.34       455
   macro avg       0.11      0.16      0.11       455
weighted avg       0.21      0.34      0.22       455



c:\Users\britt\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\britt\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\britt\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Predicting a Logistical Regression Model with Resampled Training Data 
using RandomOverSampler

In [66]:
# RandomOverSampler to resample the data 
# # Assign a random_state parameter of 42
random_oversampler_model=RandomOverSampler(random_state=42)

# Fit the original training data to the random_oversampler model
X_oversample, y_oversample = random_oversampler_model.fit_resample(X_train,y_train)

In [67]:
# Count the distinct values of the resampled labels data
y_oversample.value_counts()

2014    464
2018    464
2017    464
2016    464
2019    464
2020    464
2015    464
Name: Year of Birth, dtype: int64

In [68]:
#Logistic Regression Model
random_state_model=LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
random_state_model.fit(X_oversample,y_oversample)

# Make a prediction using the testing data
y_prediction=random_state_model.predict(X_test)

c:\Users\britt\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [69]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, y_prediction)

0.21550822089344104

In [70]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_test, y_prediction))

[[ 6  3  0  0  0  1  1]
 [12  3  0  4  4  6  3]
 [12  4  0 10  6 10  5]
 [15 13  4 17 12 11  9]
 [10  9  1 22 64 23 19]
 [11  6  2 21 42 17  7]
 [14  3  1  4  4  2  2]]


In [71]:
# Print the classification report for the model
print(classification_report(y_test, y_prediction))

              precision    recall  f1-score   support

        2014       0.07      0.55      0.13        11
        2015       0.07      0.09      0.08        32
        2016       0.00      0.00      0.00        47
        2017       0.22      0.21      0.21        81
        2018       0.48      0.43      0.46       148
        2019       0.24      0.16      0.19       106
        2020       0.04      0.07      0.05        30

    accuracy                           0.24       455
   macro avg       0.16      0.22      0.16       455
weighted avg       0.26      0.24      0.24       455

